In [1]:
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import numpy as np
import sigpy as sp
import sigpy.plot as pl
from cmcrameri import cm

from dipy.segment.mask import median_otsu
from dipy.io.image import load_nifti, save_nifti
from dipy.io.gradients import read_bvals_bvecs
import numpy as np
import nrrd
import scikit_posthocs as sp

import sys
sys.path.append("/home/ahannum/Documents/Phase/cDTI_python")
from mystic_mrpy.Data_Import.Diffusion   import *
from mystic_mrpy.Data_Sorting.Diffusion  import *
from mystic_mrpy.Diffusion.DWI  import *
from mystic_mrpy.Diffusion.Gibbs         import *
from mystic_mrpy.Diffusion.Registration  import *
from mystic_mrpy.Diffusion.Rejection     import *
from mystic_mrpy.Diffusion.Respiratory   import *
from mystic_mrpy.Diffusion.Averaging     import *
from mystic_mrpy.Diffusion.Denoising     import *
from mystic_mrpy.Diffusion.Interpolation import *
from mystic_mrpy.Diffusion.Segmentation_Matrix_DTI import *
from mystic_mrpy.Diffusion.DTI import *
from mystic_mrpy.Diffusion.cDTI import *
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import savemat

In [12]:
# Load in Mask & Magnitude image
volunteers=[1,2,3,4,5,7,8,9,10,11]
M0_mask = []
M1_mask = []
M2_mask = []

ci_m0=np.zeros((2,10,6,4,1000))
ci_m1=np.zeros((2,10,6,4,1000))
ci_m2=np.zeros((2,10,6,4,1000))



for vv in range(10):
    volunteer = str(volunteers[vv])
    print(volunteer, end = ' ')
    inpath = '/home/ahannum/Documents/Phase/Data/V00' + volunteer 
    name = inpath.split('/')
    namejoin = '/'.join(name[:-1])

    num = 0
    for num in range(3):
        mask_end   = [_ for _ in os.listdir(inpath) if _.endswith('new.nrrd')]
        mask_end.sort()
        mask_end,header =  nrrd.read(os.path.join(inpath, mask_end[num]))
        mask = mask_end
        mask = mask.astype('float')
        mask[mask==0] = np.nan
        if num ==0:
            M0_mask.append(mask)
        if num ==1:
            M1_mask.append(mask)
        if num ==2:
            M2_mask.append(mask)


    savename = os.path.join(inpath,'M0_diff_boostrap.npy')
    M0_diff = np.load(savename)

    savename = os.path.join(inpath,'M1_diff_boostrap.npy')
    M1_diff = np.load(savename)

    savename = os.path.join(inpath,'M2_diff_boostrap.npy')
    M2_diff = np.load(savename)

    for dd in range(4):
        for sl in range(6):
            for td in range(1000):
                ###############################################################################################
                im = (M0_diff[:,:,sl,dd,td,:]) * M0_mask[vv][:,:,sl][:,:,np.newaxis]
                test = im.reshape(128*128*5)
                d=test[np.logical_not(np.isnan(test))]

                bound= round(len(np.sort(d))*0.025)
                ci_m0[0,vv,sl,dd,td] = np.sort(d)[bound]
                ci_m0[1,vv,sl,dd,td] = np.sort(d)[len(d)-bound]

                im = (M1_diff[:,:,sl,dd,td,:]) * M1_mask[vv][:,:,sl][:,:,np.newaxis]
                test = im.reshape(128*128*5)
                d=test[np.logical_not(np.isnan(test))]


                bound= round(len(np.sort(d))*0.025)
                ci_m1[0,vv,sl,dd,td] = np.sort(d)[bound]
                ci_m1[1,vv,sl,dd,td] = np.sort(d)[len(d)-bound]

                im = (M2_diff[:,:,sl,dd,td,:]) * M2_mask[vv][:,:,sl][:,:,np.newaxis]
                test = im.reshape(128*128*5)
                d=test[np.logical_not(np.isnan(test))]

                bound= round(len(np.sort(d))*0.025)
                ci_m2[0,vv,sl,dd,td] = np.sort(d)[bound]
                ci_m2[1,vv,sl,dd,td] = np.sort(d)[len(d)-bound]

            




1 2 3 4 5 7 8 9 10 11 

In [14]:
ci_all = np.stack((ci_m0,ci_m1,ci_m2),axis = -1)
np.save('/home/ahannum/Documents/Phase/Data/ci_all.npy',ci_all)

In [5]:
ci_all = np.load('/home/ahannum/Documents/Phase/Data/ci_all.npy')
ci_all.dtype

dtype('float64')

In [19]:
# Plot Gz over different trigger delay times 
ci_length = ci_all[1,:,:,:,:,:] - ci_all[0,:,:,:,:,:]
ci_length_mean = np.nanmean(ci_length,axis = 0)


fig, axes = plt.subplots(3, 4, figsize=(15, 10),sharex=True,sharey = True,dpi=1000)
count = 0
slices = [0,1,2]
for sl in range(3):
    for dd in range(4):
        for ii in range(10):
            axes[sl,dd].plot(ci_length[ii,slices[sl],dd,:,:].T,color = 'lightgray',linewidth = 2)
        
        med = np.nanmedian(ci_length[:,slices[sl],dd,:,:],axis = (0,1))
        axes[sl,dd].plot(med,'o-',color = 'black',linewidth = 5,markersize = 10,alpha = 1)
        axes[sl,dd].grid(alpha=0.4)    

        axes[sl,dd].set_xticks([0,1,2])
        axes[sl,dd].set_xticklabels(['1','2','3'],size = 25)
        axes[sl,dd].tick_params(axis='y', which='major', labelsize=35)
        axes[sl,dd].set_yticks([0,math.pi,2*math.pi])
        axes[sl,dd].set_yticklabels(['$0$','$\pi$','$2\pi$'],size = 25,rotation = 0)
        axes[sl,dd].set_ylim([0,2*math.pi])
        axes[sl,dd].set_xlim([-0.3,2.3])
    
    #print(count)
    count = count +1
    



plt.subplots_adjust(wspace=0.15, hspace=0.15)
plt.show() 

#fig.savefig('/Users/arielhannum/Documents/Stanford/CMR-Setsompop/Projects/Phase/Exp1brain_dir{}_Ci_lineplot_3sl.png'.format(dd))

0
(3,)
1
(3,)
2
(3,)
3
(3,)
0
(3,)
1
(3,)
2
(3,)
3
(3,)
0
(3,)
1
(3,)
2
(3,)
3
(3,)


In [ ]:
fig.savefig('Exp1brain_dir{}_Ci_lineplot_3sl.png'.format(dd))

In [14]:
med = np.nanmedian(ci_length[:,:,,:,:],axis = 1)

In [15]:
med.shape

(3,)